In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
# Initialize thevariables
cols = 4
rows = 3

gamma = 0.9

R = [[-0.04,-0.04,-0.04,1],
     [-0.04,-0.04,-0.04,-1],
     [-0.04,-0.04,-0.04,-0.04]]

U = [[0,0,0,1],
     [0,0,0,-1],
     [0,0,0,0],
     [0,0,0,0]]

p_intended = 0.8
p_slip = 0.1

actions = {
    'up':    (0, -1),
    'down':  (0, 1),
    'left':  (-1, 0),
    'right': (1, 0)
}

In [3]:
U = [[0.0 for x in range(cols)] for y in range(rows)]

In [4]:
# Initialize the Terminal States
U[1][3] = -1
U[0][3] = 1

In [5]:
def in_grid(x,y):
  return 0<x<cols and 0<y<rows